### Metrics-6b: Variations on fastai.Tabular
Study the change in perf for a variety of alterations to the modelling with fastai.Tabular

Variations:
 - concise X vs. all pts X vs. all pix + pts X
 - normalized x vs unnormalized
 - normalized y vs unnormalized
 - topleft vs center
 - number of fit epochs

In [1]:
import os, sys
import copy as copyroot
import pandas as pd
from IPython.display import display
from matplotlib import pyplot as plt

from fastai2.basics import *
from fastai2.tabular.all import *

from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.metrics import r2_score

%load_ext autoreload
%autoreload 2

from module.mnist_helpers import build_df, build_tabular_df, build_dls
from module.mnist_metrics import metrics_df

In [35]:
path = untar_data(URLs.MNIST_TINY)
X, Y = build_tabular_df(path)

In [36]:
dls_tl = build_dls(target='topleft')

point_t = dls_tl.transform[1][1]
scale_t = dls_tl.after_item
def my_scale(x): return scale_t(point_t(x))

Y = pd.DataFrame(
        torch.cat( ( my_scale(Y.iloc[:,1:3]), 
                     my_scale(Y.iloc[:,3:5]) ), axis=1),
        columns = Y.columns[1:])

In [38]:
y_tl_cols = ['point_topleft_x', 'point_topleft_y']
y_cr_cols = ['point_center_x',  'point_center_y']

In [39]:
restrict_cols = ["pts22_5","pts22_29","pts22_21","pts11_0",
                "pts12_4", "pts11_2"]
pts_cols      = [col for col in X.columns if 'pts' in str(col)]
all_cols      = [col for col in X.columns]

In [40]:
def grid(d):
    ret = [{}]
    for param_name, values in d.items():
        old_ret = ret.copy()
        new_ret = []
        for value in values:
            for item in old_ret:
                item = item.copy()
                item[param_name] = value
                new_ret.append(item)
                ret = new_ret.copy()
    return ret

In [47]:
d_params = {'target_ind':[0,1],
            'x_sz_enum': [0,1,2],
            'x_norm':    [True, False],
            'y_norm':    [True, False],
            'epochs':    [10,40],
           }

# quick debug mode
# d_params = {'target_ind':[0,1],
#             'x_sz_enum': [0,1,2],
#             'x_norm':    [True, False],
#             'y_norm':    [True, False],
#             'epochs':    [3],
#            }

In [48]:
params = grid(d_params)
print(len(params))
# params[:3]

48


In [ ]:
t0 = time.time()
ledger = []
for param in grid(d_params):
    
    y_cols = [y_tl_cols, y_cr_cols][param['target_ind']]
    
    x_cols = [restrict_cols, pts_cols, all_cols][param['x_sz_enum']]
        
    data = pd.concat((X[x_cols], Y[y_cols]), axis=1)
    
    epochs = param['epochs']
    
    procs =  []
    if param['x_norm']: procs += [Normalize]
    
    ss = None
    if param['y_norm']:
        ss = StandardScaler()
        ss.fit(data[y_cols])
        data[y_cols] = ss.transform(data[y_cols])
        
    dls = TabularDataLoaders.from_df(data, 
                                     path='.', 
                                     y_names=y_cols,
                                     procs=procs)
    
    learn = tabular_learner(dls,)
    
    with learn.no_logging():
        learn.fit(epochs)
    
    s_details =   ''
    s_details += f"x={['restrict', 'pts', 'all'][param['x_sz_enum']]} "
    s_details += f"y_norm={param['y_norm']} "
    s_details += f"x_norm={param['x_norm']} "
    s_details += f"epochs={param['epochs']} "
    
    s_target = ["topleft", "center"][param["target_ind"]]
    
    metrics = metrics_df(learn,
                         s_model="fastTab1.2",
                         s_details=s_details,
                         s_target=s_target,
                         y_scaler=ss)
    
    ledger.append(metrics)

print(f"{round(time.time() - t0, 0)} secs")

#### Join Tables

In [53]:
df = pd.concat(ledger)
print(df.shape)
df.head(2)

(96, 11)


,model,details,target,split,mse,mae,r2,dist_avg,dist_r2,sqdist_avg,sqdist_r2
0,fastTab1.2,x=restrict y_norm=True x_norm=True epochs=10,topleft,valid,0.000275,0.010506,0.995243,0.018657,0.920853,0.000549,0.992777
1,fastTab1.2,x=restrict y_norm=True x_norm=True epochs=10,topleft,train,0.000276,0.010818,0.994929,0.019155,0.916686,0.000552,0.992422


#### Create New Cols from details

In [62]:
feats = ['x_cols','y_norm', 'x_norm', 'epochs']
for i, name in enumerate(feats):
    
    df[name] = df['details'].apply(
                    lambda s: s.split(' ')[i].split('=')[1])

In [90]:
cols = []
cols += list(df.columns[:3])
cols += list(df.columns[-4:])
cols += [e for e in df.columns if e not in cols]
df = df[cols]

#### Pivot on `x_cols`
Still have problems with all_cols...discard these from analysis

In [77]:
tmp = df[(df['target'] == 'topleft') & (df['split'] == 'valid')]

In [81]:
tmp.groupby('x_cols').agg(['min','mean','max'])['mse'].round(4)

,min,mean,max
x_cols,,,
all,1951.0884,1.243754e+20,7.619093e+20
pts,0.0003,1.600000e-03,3.400000e-03
restrict,0.0001,9.000000e-04,2.200000e-03


In [82]:
tmp = df[ (  (df['target'] == 'topleft') 
           & (df['split']  == 'valid')
           & (df['x_cols'] != 'all')    )]

In [84]:
for feat in feats:
    display(tmp.groupby(feat).agg(['min', 'mean','max'])['mse'].round(4))

,min,mean,max
x_cols,,,
pts,0.0003,0.0016,0.0034
restrict,0.0001,0.0009,0.0022


,min,mean,max
y_norm,,,
False,0.0007,0.0022,0.0034
True,0.0001,0.0003,0.0005


,min,mean,max
x_norm,,,
False,0.0001,0.0011,0.0026
True,0.0002,0.0013,0.0034


,min,mean,max
epochs,,,
10,0.0001,0.0011,0.0029
40,0.0002,0.0013,0.0034


Surprsing: epochs=10 outperfroms 40, and x_norm hurts us.

turning on y_norm has the biggest effect

In [85]:
for feat in feats:
    display(tmp.groupby(feat).agg(['min', 'mean','max'])['r2'].round(4))

,min,mean,max
x_cols,,,
pts,0.8149,0.9207,0.9932
restrict,0.8210,0.9471,0.9965


,min,mean,max
y_norm,,,
False,0.8149,0.8745,0.9428
True,0.9889,0.9932,0.9965


,min,mean,max
x_norm,,,
False,0.8210,0.9319,0.9945
True,0.8149,0.9358,0.9965


,min,mean,max
epochs,,,
10,0.8149,0.9390,0.9952
40,0.8210,0.9287,0.9965


We also see that going from x_cols restrict -> pts has a negative effect, but not much.

In [97]:
tmp = df[ (  (df['split']  == 'valid')
           & (df['x_cols'] != 'all')    )]

for feat in feats:
    display(tmp.groupby(['target', feat]).agg(['min', 'mean','max'])['r2'].round(4))

min    mean     max
target  x_cols                          
center  pts       0.6683  0.8858  0.9724
        restrict  0.2343  0.4347  0.5509
topleft pts       0.8149  0.9207  0.9932
        restrict  0.8210  0.9471  0.9965

min    mean     max
target  y_norm                        
center  False   0.2343  0.5880  0.8663
        True    0.4525  0.7325  0.9724
topleft False   0.8149  0.8745  0.9428
        True    0.9889  0.9932  0.9965

min    mean     max
target  x_norm                        
center  False   0.2343  0.6623  0.9724
        True    0.3234  0.6582  0.9718
topleft False   0.8210  0.9319  0.9945
        True    0.8149  0.9358  0.9965

min    mean     max
target  epochs                        
center  10      0.4525  0.6637  0.9724
        40      0.2343  0.6568  0.9718
topleft 10      0.8149  0.9390  0.9952
        40      0.8210  0.9287  0.9965

For the center task, it's actually better to have the all features frompts in x_cols (which makes sense because there's no perfect predictor feature.) So feature avaialbility is a t a premium on the center task, followed by y_norm'ing. In this case, x_norm'ing does sometimes help, especially with bad fits.

#### Display Topleft + Valid

In [92]:
tmp = df[(df['target'] == 'topleft') & (df['split'] == 'valid')]
tmp.sort_values(by='mse', ascending=True).round(4)

,model,details,target,x_cols,y_norm,x_norm,epochs,split,mse,mae,r2,dist_avg,dist_r2,sqdist_avg,sqdist_r2
0,fastTab1.2,x=restrict y_norm=True x_norm=False epochs=10,topleft,restrict,True,False,10,valid,1.000000e-04,7.700000e-03,9.945000e-01,1.230000e-02,9.460000e-01,2.000000e-04,9.974000e-01
0,fastTab1.2,x=restrict y_norm=True x_norm=True epochs=40,topleft,restrict,True,True,40,valid,2.000000e-04,8.500000e-03,9.965000e-01,1.430000e-02,9.404000e-01,3.000000e-04,9.959000e-01
0,fastTab1.2,x=restrict y_norm=True x_norm=True epochs=10,topleft,restrict,True,True,10,valid,3.000000e-04,1.050000e-02,9.952000e-01,1.870000e-02,9.209000e-01,5.000000e-04,9.928000e-01
0,fastTab1.2,x=pts y_norm=True x_norm=True epochs=10,topleft,pts,True,True,10,valid,3.000000e-04,1.100000e-02,9.932000e-01,1.870000e-02,9.217000e-01,6.000000e-04,9.927000e-01
0,fastTab1.2,x=pts y_norm=True x_norm=False epochs=40,topleft,pts,True,False,40,valid,3.000000e-04,1.240000e-02,9.927000e-01,2.040000e-02,9.203000e-01,6.000000e-04,9.937000e-01
0,fastTab1.2,x=pts y_norm=True x_norm=True epochs=40,topleft,pts,True,True,40,valid,3.000000e-04,1.180000e-02,9.921000e-01,1.970000e-02,9.175000e-01,6.000000e-04,9.924000e-01
0,fastTab1.2,x=restrict y_norm=True x_norm=False epochs=40,topleft,restrict,True,False,40,valid,4.000000e-04,1.230000e-02,9.929000e-01,2.150000e-02,9.067000e-01,7.000000e-04,9.901000e-01
0,fastTab1.2,x=pts y_norm=True x_norm=False epochs=10,topleft,pts,True,False,10,valid,5.000000e-04,1.460000e-02,9.889000e-01,2.430000e-02,9.081000e-01,9.000000e-04,9.903000e-01
0,fastTab1.2,x=restrict y_norm=False x_norm=False epochs=10,topleft,restrict,False,False,10,valid,7.000000e-04,1.990000e-02,9.428000e-01,3.130000e-02,8.634000e-01,1.300000e-03,9.817000e-01
0,fastTab1.2,x=restrict y_norm=False x_norm=True epochs=40,topleft,restrict,False,True,40,valid,1.400000e-03,2.870000e-02,8.963000e-01,4.410000e-02,8.142000e-01,2.900000e-03,9.622000e-01


#### Display Center + Valid

In [95]:
tmp = df[(df['target'] == 'center') & (df['split'] == 'valid')]
tmp.sort_values(by='mse', ascending=True).round(4)

,model,details,target,x_cols,y_norm,x_norm,epochs,split,mse,mae,r2,dist_avg,dist_r2,sqdist_avg,sqdist_r2
0,fastTab1.2,x=pts y_norm=True x_norm=True epochs=40,center,pts,True,True,40,valid,1.000000e-04,8.700000e-03,9.718000e-01,1.440000e-02,8.628000e-01,3.000000e-04,9.797000e-01
0,fastTab1.2,x=pts y_norm=True x_norm=True epochs=10,center,pts,True,True,10,valid,1.000000e-04,8.900000e-03,9.669000e-01,1.470000e-02,8.519000e-01,3.000000e-04,9.753000e-01
0,fastTab1.2,x=pts y_norm=True x_norm=False epochs=10,center,pts,True,False,10,valid,2.000000e-04,8.900000e-03,9.724000e-01,1.440000e-02,8.683000e-01,3.000000e-04,9.781000e-01
0,fastTab1.2,x=pts y_norm=True x_norm=False epochs=40,center,pts,True,False,40,valid,2.000000e-04,8.400000e-03,9.691000e-01,1.370000e-02,8.704000e-01,3.000000e-04,9.771000e-01
0,fastTab1.2,x=pts y_norm=False x_norm=True epochs=40,center,pts,False,True,40,valid,8.000000e-04,2.290000e-02,8.663000e-01,3.560000e-02,6.779000e-01,1.700000e-03,8.859000e-01
0,fastTab1.2,x=pts y_norm=False x_norm=False epochs=40,center,pts,False,False,40,valid,9.000000e-04,2.290000e-02,8.582000e-01,3.590000e-02,6.653000e-01,1.700000e-03,8.737000e-01
0,fastTab1.2,x=pts y_norm=False x_norm=False epochs=10,center,pts,False,False,10,valid,1.300000e-03,2.710000e-02,8.138000e-01,4.310000e-02,6.084000e-01,2.600000e-03,8.256000e-01
0,fastTab1.2,x=restrict y_norm=True x_norm=False epochs=10,center,restrict,True,False,10,valid,2.000000e-03,2.940000e-02,4.954000e-01,5.150000e-02,4.741000e-01,4.000000e-03,6.541000e-01
0,fastTab1.2,x=pts y_norm=False x_norm=True epochs=10,center,pts,False,True,10,valid,2.100000e-03,3.310000e-02,6.683000e-01,5.310000e-02,5.111000e-01,4.200000e-03,7.043000e-01
0,fastTab1.2,x=restrict y_norm=True x_norm=True epochs=40,center,restrict,True,True,40,valid,2.100000e-03,3.220000e-02,5.509000e-01,5.360000e-02,4.772000e-01,4.300000e-03,6.756000e-01


#### Save

In [94]:
df.to_csv('assets/metrics6b-fasttab-grid.csv', index=False)

#### Plots
Maybe with log scaling?